# Prepare Datasets for Benchmarking Tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2022-10-05  


## Dataset1. Enzyme None-enzyme Dataset
The enzyme dataset is consists of two parts: <u>a training set</u> and <u>a testing set</u>.   
The training set is from snapshot Feb-2018 and ***excludes*** those <u>deleted items</u> and <u>sequences changed items</u> in snapshot Feb-2022.    
The training set is consists of ***469,134*** records, of which ***222,567*** are enzymes, and ***246,567*** are none-enzymes.   
The testing set is from snapshot Feb-2022 and excludes these items that appeared in snapshot Feb-2018.   
The testing set is consists of ***10,614*** records, of which ***5111*** are enzymes, and ***5503*** are none-enzymes.   
Unlike previous works,  we did not filter any sequences in terms of length and homology to make the data more inclusive. We make a label for each sequence, 1 for enzyme and 0 for none-enzyme.   

## Dataset2. Enzyme Function Quantity Dataset
The enzyme quantity dataset only contains enzyme data, contain ***222,567*** records. The function quantity ranges from 1 to 8.

## Dataset 3: EC Dataset

The EC dataset consists of 227,678 enzyme records, 222,567 are training-set, and the rest 5111 are testing-set, covering 6,031 EC numbers. Up to Feb 2022, ***cmopared with [ExplorEnz](https://www.enzyme-database.org/stats.php) CURRENT EC = 6674***, there still exist 643 EC numbers that the model can not handle in the benchmarking. Thus, we exclude the sequences with these 267 EC numbers in the evaluation process. But, this problem can be resolved in the production scenario because we use the entire data from Swiss-Prot. Now the EC coverage is 6,031 and can be automatically extended, for the training is real-time based on the publication of Swiss-Prot every 8 weeks. 

## 1. Import packages

In [5]:
import numpy as np
import pandas as pd
import sys,os
from tqdm import tqdm
import config as cfg
from functools import reduce

from tools import filetool as ftool
from tools import exact_ec_from_uniprot as exactec
from tools import funclib
from tools import minitools as mtool
# from tools import embedding_esm as esmebd
from tools import embdding_onehot as onehotebd

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)


%load_ext autoreload
%autoreload 2
print(f'{os.getcwd()}\\')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Erfan\Downloads\Saeedeh\ECRECer\


## 2. Download rawdata from unisprot

> IF first time run pls uncomment the cell below

In [6]:
#snapshot 2018-02
ftool.wget(download_url=cfg.URL_SPROT_SNAP201802, save_file=cfg.FILE_SPROT_SNAP201802)

#snapshot 2019-12
ftool.wget(download_url=cfg.URL_SPROT_SNAP201902, save_file=cfg.FILE_SPROT_SNAP201902)

#snapshot 2020-06
ftool.wget(download_url=cfg.URL_SPROT_SNAP202006, save_file=cfg.FILE_SPROT_SNAP202006)

#snapshot 2021-02
ftool.wget(download_url=cfg.URL_SPROT_SNAP202102, save_file=cfg.FILE_SPROT_SNAP202102)

#snapshot 2022-02
ftool.wget(download_url=cfg.URL_SPROT_SNAP202202, save_file=cfg.FILE_SPROT_SNAP202202)

wget not installed
wget not installed
wget not installed
wget not installed
wget not installed


## 3. Extract records from rawdata

In [26]:
cmd_array = [
    # 2018 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP201802} -C {cfg.DIR_UNIPROT}',
    f'move {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2018.data.gz', 
    f'del /Q /F {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',
    
        
    # 2019 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP201902} -C {cfg.DIR_UNIPROT}',
    f'move {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2019.data.gz', 
    f'del /Q /F {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2020 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202006} -C {cfg.DIR_UNIPROT}',
    f'move {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2020.data.gz', 
    f'del /Q /F {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2021 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202102} -C {cfg.DIR_UNIPROT}',
    f'move {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2021.data.gz', 
    f'del /Q /F {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2022 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202202} -C {cfg.DIR_UNIPROT}',
    f'move {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2022.data.gz', 
    f'del /Q /F {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz'
]

[os.system(item) for item in cmd_array]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
# snapshot 2018-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2018.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2018.tsv')

# #snapshot 2019-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2019.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2019.tsv')

#snapshot 2020-06
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2020.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2020.tsv')

#snapshot 2021-06
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2021.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2021.tsv')

# #snapshot 2022-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2022.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2022.tsv')


0it [00:00, ?it/s]
401it [00:00, 4009.76it/s]
802it [00:00, 2782.48it/s]
1108it [00:00, 2884.55it/s]
1513it [00:00, 3275.23it/s]
1966it [00:00, 3676.25it/s]
2347it [00:00, 3121.22it/s]
2679it [00:00, 2963.02it/s]
2989it [00:00, 2952.16it/s]
3294it [00:01, 2780.10it/s]
3579it [00:01, 2754.92it/s]
3859it [00:01, 2557.23it/s]
4120it [00:01, 2097.88it/s]
4344it [00:01, 1968.62it/s]
4556it [00:01, 2004.39it/s]
4765it [00:01, 1971.13it/s]
5106it [00:01, 2345.93it/s]
5451it [00:02, 2638.61it/s]
5767it [00:02, 2776.13it/s]
6053it [00:02, 2784.21it/s]
6337it [00:02, 2626.52it/s]
6605it [00:02, 2247.51it/s]
6860it [00:02, 2318.09it/s]
7102it [00:02, 2161.91it/s]
7327it [00:02, 2124.28it/s]
7754it [00:02, 2684.16it/s]
8240it [00:03, 3270.32it/s]
8580it [00:03, 3216.50it/s]
8911it [00:03, 2930.46it/s]
9215it [00:03, 2761.44it/s]
9500it [00:03, 2699.81it/s]
9778it [00:03, 2706.79it/s]
10053it [00:03, 2339.95it/s]
10297it [00:03, 2188.55it/s]
10605it [00:04, 2411.28it/s]
10858it [00:04, 2436.33it/s

144453it [00:56, 2689.04it/s]
144748it [00:56, 2758.11it/s]
145229it [00:57, 3366.28it/s]
145568it [00:57, 3131.16it/s]
145886it [00:57, 2669.28it/s]
146261it [00:57, 2945.24it/s]
146574it [00:57, 2986.70it/s]
146884it [00:57, 2898.06it/s]
147323it [00:57, 3298.56it/s]
147662it [00:57, 3164.16it/s]
147986it [00:58, 2971.46it/s]
148290it [00:58, 2919.17it/s]
148587it [00:58, 2683.88it/s]
148861it [00:58, 2453.42it/s]
149144it [00:58, 2542.84it/s]
149404it [00:58, 2498.20it/s]
149658it [00:58, 2417.79it/s]
149903it [00:58, 2350.00it/s]
150187it [00:58, 2477.05it/s]
150437it [00:59, 2256.00it/s]
150765it [00:59, 2528.98it/s]
151025it [00:59, 2486.85it/s]
151319it [00:59, 2605.45it/s]
151584it [00:59, 2603.32it/s]
151912it [00:59, 2789.58it/s]
152194it [00:59, 2766.29it/s]
152474it [00:59, 2768.06it/s]
152753it [00:59, 2516.50it/s]
153010it [01:00, 2291.72it/s]
153246it [01:00, 2172.80it/s]
153513it [01:00, 2296.44it/s]
153748it [01:00, 2244.09it/s]
154064it [01:00, 2487.46it/s]
154410it [

295218it [01:52, 2140.70it/s]
295438it [01:52, 2099.52it/s]
295758it [01:52, 2402.34it/s]
296004it [01:52, 2320.34it/s]
296240it [01:53, 2113.32it/s]
296457it [01:53, 2041.66it/s]
296699it [01:53, 2137.05it/s]
297006it [01:53, 2393.72it/s]
297307it [01:53, 2560.47it/s]
297623it [01:53, 2724.62it/s]
297937it [01:53, 2836.85it/s]
298286it [01:53, 3019.01it/s]
298591it [01:53, 2900.23it/s]
298884it [01:53, 2867.65it/s]
299186it [01:54, 2911.48it/s]
299479it [01:54, 2883.27it/s]
299769it [01:54, 2613.74it/s]
300227it [01:54, 3143.47it/s]
300634it [01:54, 3403.21it/s]
300982it [01:54, 3222.56it/s]
301312it [01:54, 2876.55it/s]
301611it [01:54, 2803.77it/s]
301899it [01:55, 2560.62it/s]
302205it [01:55, 2687.64it/s]
302503it [01:55, 2758.27it/s]
302785it [01:55, 2672.48it/s]
303057it [01:55, 2656.34it/s]
303332it [01:55, 2675.06it/s]
303602it [01:55, 2622.39it/s]
303946it [01:55, 2847.05it/s]
304233it [01:55, 2845.46it/s]
304606it [01:55, 3094.10it/s]
304917it [01:56, 2758.97it/s]
305201it [

469356it [02:48, 2245.07it/s]
469617it [02:48, 2342.76it/s]
469854it [02:48, 2350.62it/s]
470102it [02:48, 2388.40it/s]
470435it [02:49, 2657.81it/s]
470780it [02:49, 2891.86it/s]
471071it [02:49, 2830.07it/s]
471383it [02:49, 2914.92it/s]
471708it [02:49, 3013.43it/s]
472011it [02:49, 2670.39it/s]
472286it [02:49, 2507.56it/s]
472544it [02:49, 2513.47it/s]
472859it [02:49, 2688.12it/s]
473181it [02:50, 2829.95it/s]
473516it [02:50, 2978.84it/s]
473818it [02:50, 2898.25it/s]
474111it [02:50, 2530.21it/s]
474374it [02:50, 2398.84it/s]
474622it [02:50, 2407.39it/s]
474939it [02:50, 2606.43it/s]
475206it [02:50, 2595.25it/s]
475470it [02:50, 2564.49it/s]
475730it [02:51, 2397.69it/s]
475974it [02:51, 2243.31it/s]
476203it [02:51, 2249.77it/s]
476532it [02:51, 2529.57it/s]
476790it [02:51, 2246.10it/s]
477036it [02:51, 2302.18it/s]
477287it [02:51, 2352.48it/s]
477602it [02:51, 2568.41it/s]
477939it [02:51, 2788.89it/s]
478242it [02:52, 2850.28it/s]
478531it [02:52, 2631.77it/s]
478800it [

finished use time 194.625 s


...

129489it [00:58, 2108.37it/s]
129706it [00:58, 2044.56it/s]
129979it [00:58, 2219.43it/s]
130274it [00:58, 2418.90it/s]
130529it [00:58, 2448.87it/s]
130927it [00:58, 2877.95it/s]
131220it [00:59, 2884.67it/s]
131512it [00:59, 2572.27it/s]
131778it [00:59, 2582.03it/s]
132043it [00:59, 2317.92it/s]
132408it [00:59, 2644.93it/s]
132683it [00:59, 2141.27it/s]
132967it [00:59, 2306.03it/s]
133219it [00:59, 2354.08it/s]
133469it [01:00, 2037.35it/s]
133689it [01:00, 1867.86it/s]
133889it [01:00, 1868.19it/s]
134199it [01:00, 2171.62it/s]
134428it [01:00, 2135.33it/s]
134672it [01:00, 2216.72it/s]
134940it [01:00, 2338.24it/s]
135201it [01:00, 2414.97it/s]
135548it [01:00, 2718.25it/s]
135825it [01:01, 2196.26it/s]
136064it [01:01, 2122.35it/s]
136290it [01:01, 2109.30it/s]
136549it [01:01, 2228.79it/s]
136816it [01:01, 2348.37it/s]
137058it [01:01, 2304.07it/s]
137370it [01:01, 2532.11it/s]
137733it [01:01, 2845.39it/s]
138103it [01:01, 3083.93it/s]
138474it [01:02, 3257.57it/s]
138804it [

260910it [01:54, 2401.82it/s]
261182it [01:54, 2486.75it/s]
261468it [01:55, 2595.14it/s]
261775it [01:55, 2726.87it/s]
262082it [01:55, 2819.79it/s]
262366it [01:55, 2776.48it/s]
262645it [01:55, 2670.75it/s]
262914it [01:55, 2676.27it/s]
263183it [01:55, 2656.82it/s]
263467it [01:55, 2710.24it/s]
263747it [01:55, 2728.89it/s]
264021it [01:56, 2623.74it/s]
264285it [01:56, 2583.71it/s]
264545it [01:56, 2480.85it/s]
264795it [01:56, 2424.85it/s]
265039it [01:56, 2362.12it/s]
265276it [01:56, 2351.29it/s]
265512it [01:56, 2269.39it/s]
265740it [01:56, 2078.53it/s]
265970it [01:56, 2132.51it/s]
266186it [01:57, 1731.44it/s]
266373it [01:57, 1502.46it/s]
266547it [01:57, 1553.08it/s]
266790it [01:57, 1763.85it/s]
266979it [01:57, 1755.60it/s]
267239it [01:57, 1979.84it/s]
267538it [01:57, 2251.50it/s]
267883it [01:57, 2581.62it/s]
268149it [01:57, 2560.06it/s]
268416it [01:58, 2584.20it/s]
268685it [01:58, 2607.33it/s]
268966it [01:58, 2666.58it/s]
269235it [01:58, 2596.99it/s]
269497it [

407990it [02:52, 4064.16it/s]
408443it [02:52, 4201.02it/s]
408904it [02:52, 4309.88it/s]
409339it [02:52, 4113.87it/s]
409815it [02:52, 4299.46it/s]
410249it [02:52, 3679.64it/s]
410768it [02:52, 4075.10it/s]
411274it [02:52, 4331.70it/s]
411778it [02:53, 4517.06it/s]
412242it [02:53, 4463.10it/s]
412697it [02:53, 4412.36it/s]
413144it [02:53, 4416.01it/s]
413590it [02:53, 4415.95it/s]
414066it [02:53, 4503.49it/s]
414587it [02:53, 4697.10it/s]
415105it [02:53, 4839.36it/s]
415624it [02:53, 4929.47it/s]
416171it [02:54, 5089.49it/s]
416698it [02:54, 5143.12it/s]
417229it [02:54, 5177.62it/s]
417756it [02:54, 5205.13it/s]
418277it [02:54, 5156.80it/s]
418794it [02:54, 4968.69it/s]
419302it [02:54, 5000.95it/s]
419804it [02:54, 4977.08it/s]
420303it [02:54, 4908.72it/s]
420795it [02:54, 4467.53it/s]
421250it [02:55, 4334.83it/s]
421689it [02:55, 4101.75it/s]
422105it [02:55, 3906.41it/s]
422500it [02:55, 3756.62it/s]
422879it [02:55, 3725.52it/s]
423255it [02:55, 3725.13it/s]
423629it [

556180it [03:47, 3022.88it/s]
556670it [03:47, 3531.45it/s]
557032it [03:48, 3446.96it/s]
557383it [03:48, 3306.93it/s]
557743it [03:48, 3387.90it/s]
558086it [03:48, 3288.41it/s]
558440it [03:48, 3358.83it/s]
558819it [03:48, 3472.23it/s]
559169it [03:48, 3382.47it/s]
559522it [03:48, 3415.28it/s]
559906it [03:48, 3538.02it/s]
560310it [03:48, 3674.90it/s]
560712it [03:49, 3776.21it/s]
561091it [03:49, 3650.89it/s]
561458it [03:49, 3583.66it/s]
561880it [03:49, 3756.60it/s]
562338it [03:49, 3996.52it/s]
562740it [03:49, 3899.94it/s]
563132it [03:49, 3817.13it/s]
563515it [03:49, 3479.57it/s]
563869it [03:49, 3394.54it/s]
564246it [03:50, 3497.28it/s]
564600it [03:50, 3138.32it/s]
564978it [03:50, 3308.14it/s]
565317it [03:50, 2378.07it/s]
565596it [03:50, 2021.46it/s]
565833it [03:50, 2014.73it/s]
566059it [03:51, 1913.46it/s]
566267it [03:51, 1709.23it/s]
566451it [03:51, 1597.78it/s]
566619it [03:51, 1530.61it/s]
566777it [03:51, 1517.41it/s]
566952it [03:51, 1574.64it/s]
567161it [

finished use time 231.781 s


## 4. Load records & Drop Duplicates

In [28]:
#加载数据并转换时间格式
sprot2018 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2018.tsv', sep='\t',header=0) #读入文件
sprot2018 = mtool.convert_DF_dateTime(inputdf = sprot2018)

sprot2019 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2019.tsv', sep='\t',header=0) #读入文件
sprot2019 = mtool.convert_DF_dateTime(inputdf = sprot2019)

sprot2020 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2020.tsv', sep='\t',header=0) #读入文件
sprot2020 = mtool.convert_DF_dateTime(inputdf = sprot2020)

sprot2021 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2021.tsv', sep='\t',header=0) #读入文件
sprot2021 = mtool.convert_DF_dateTime(inputdf = sprot2021)

sprot2022 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2022.tsv', sep='\t',header=0) #读入文件
sprot2022 = mtool.convert_DF_dateTime(inputdf = sprot2022)


#Drop Duplicates
sprot2018.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2018.reset_index(drop=True, inplace=True)

sprot2019.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2019.reset_index(drop=True, inplace=True)

sprot2020.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2020.reset_index(drop=True, inplace=True)

sprot2021.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2021.reset_index(drop=True, inplace=True)

sprot2022.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2022.reset_index(drop=True, inplace=True)

In [29]:
sprot2018.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,Q3J1A3,LHB1_RHOS4,False,False,0,-,0,1986-07-21,2007-01-23,2017-10-25,MADKSDLGYTGLTDEQAQELHSVYMSGLWLFSAVAIVAHLAVYIWRPWF,49
1,P02157,MYG_MELME,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNVWGKVEADLAGHGQEVLIRLFKGHPETLEKFDK...,154
2,P02178,MYG_MEGNO,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,154


In [30]:
sprot2020.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P03711,SCAF_LAMBD,True,False,1,3.4.21.-,3,1986-07-21,1986-07-21,2020-12-02,MTAELRNLPHIASMAFNEPLMLEPAYARVFFCALAGQLGISSLTDA...,439
1,P01027,CO3_MOUSE,False,False,0,-,0,1986-07-21,2011-07-27,2020-12-02,MGPASGSQLLVLLLLLASSPLALGIPMYSIITPNVLRLESEETIVL...,1663
2,P02706,ASGR1_RAT,False,False,0,-,0,1986-07-21,2007-01-23,2020-12-02,MTKDYQDFQHLDNENDHHQLQRGPPPAPRLLQRLCSGFRLFLLSLG...,284


In [31]:
sprot2022.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P00250,FER_APHSA,False,False,0,-,0,1986-07-21,2007-01-23,2022-05-25,MASYKVTLKTPDGDNVITVPDDEYILDVAEEEGLDLPYSCRAGACS...,97
1,P03420,FUS_HRSVA,False,False,0,-,0,1986-07-21,1986-07-21,2022-05-25,MELLILKANAITTILTAVTFCFASGQNITEEFYQSTCSAVSKGYLS...,574
2,P0ACF7,DBHB_SHIFL,False,False,0,-,0,1986-07-21,1986-07-21,2022-05-25,MNKSQLIDKIAAGADISKAAAGRALDAIIASVTESLKEGDDVALVG...,90


## 5. Preprocessing
### 5.1 format EC

In [41]:
#sprot2018
sprot2018['ec_number'] = sprot2018.ec_number.apply(lambda x: mtool.format_ec(x))
sprot2018['ec_number'] = sprot2018.ec_number.apply(lambda x: mtool.specific_ecs(x))
sprot2018['functionCounts'] = sprot2018.ec_number.apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2018 finished')

#sprot2019
sprot2019['ec_number'] = sprot2019.ec_number.apply(lambda x: mtool.format_ec(x))
sprot2019['ec_number'] = sprot2019.ec_number.apply(lambda x: mtool.specific_ecs(x))
sprot2019['functionCounts'] = sprot2019.ec_number.apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2019 finished')

#sprot2020
sprot2020['ec_number'] = sprot2020.ec_number.apply(lambda x: mtool.format_ec(x))
sprot2020['ec_number'] = sprot2020.ec_number.apply(lambda x: mtool.specific_ecs(x))
sprot2020['functionCounts'] = sprot2020.ec_number.apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2020 finished')

#sprot2021
sprot2021['ec_number'] = sprot2021.ec_number.apply(lambda x: mtool.format_ec(x))
sprot2021['ec_number'] = sprot2021.ec_number.apply(lambda x: mtool.specific_ecs(x))
sprot2021['functionCounts'] = sprot2021.ec_number.apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2021 finished')


#sprot2022
sprot2022['ec_number'] = sprot2022.ec_number.apply(lambda x: mtool.format_ec(x))
sprot2022['ec_number'] = sprot2022.ec_number.apply(lambda x: mtool.specific_ecs(x))
sprot2022['functionCounts'] = sprot2022.ec_number.apply(lambda x: 0 if x=='-'  else len(x.split(',')))

print('sprot 2022 finished')

sprot 2018 finished
sprot 2019 finished
sprot 2020 finished
sprot 2021 finished
sprot 2022 finished


In [42]:
sprot2018.to_feather(cfg.DIR_UNIPROT + '/snap201802.feather')
sprot2019.to_feather(cfg.DIR_UNIPROT + '/snap201902.feather')
sprot2020.to_feather(cfg.DIR_UNIPROT + '/snap202006.feather')
sprot2021.to_feather(cfg.DIR_UNIPROT + '/snap202102.feather')
sprot2022.to_feather(cfg.DIR_UNIPROT + '/snap202202.feather')

### 5.2 Split Tain Test

In [43]:
sprot2018

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,Q3J1A3,LHB1_RHOS4,False,False,0,-,0,1986-07-21,2007-01-23,2017-10-25,MADKSDLGYTGLTDEQAQELHSVYMSGLWLFSAVAIVAHLAVYIWRPWF,49
1,P02157,MYG_MELME,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNVWGKVEADLAGHGQEVLIRLFKGHPETLEKFDK...,154
2,P02178,MYG_MEGNO,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,154
3,P02194,MYG_MACRU,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNIWGKVETDEGGHGKDVLIRLFKGHPETLEKFDK...,154
4,P01915,HB22_MOUSE,False,False,0,-,0,1986-07-21,1986-07-21,2017-10-25,MVWLPRVPCVAAVILLLTVLSPPVALVRDTRPRFLEYVTSECHFYN...,264
...,...,...,...,...,...,...,...,...,...,...,...,...
469129,Q21221,AHO3_CAEEL,True,False,1,3.1.2.22,4,2018-02-28,2004-11-23,2018-02-28,MSSGAPSGSSMSSTPGSPPPRAGGPNSVSFKDLCCLFCCPPFPSSI...,332
469130,Q6QJ72,PDL2_ARATH,True,False,1,4.2.1.96,4,2018-02-28,2004-07-05,2018-02-28,MSRLLLPKLFSISRTQVPAASLFNNLYRRHKRFVHWTSKMSTDSVR...,187
469131,C0HL68,ES1GA_ODOGR,False,False,0,-,0,2018-02-28,2018-02-28,2018-02-28,GLFSKPAGKGIKNLIPKGVKHIGKEVGKDVIRTGIDVAGCKIKGEC,46
469132,C0HK74,VKT3_HETMG,False,False,0,-,0,2018-02-28,2018-02-28,2018-02-28,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...,56


In [44]:
train = sprot2018.iloc[:,np.r_[0,2:8,10:12]]

test_2019 = sprot2019.iloc[:,np.r_[0,2:8,10:12]]
test_2020 = sprot2020.iloc[:,np.r_[0,2:8,10:12]]
test_2021 = sprot2021.iloc[:,np.r_[0,2:8,10:12]]
test_2022 = sprot2022.iloc[:,np.r_[0,2:8,10:12]]

test_2019 =test_2019[~test_2019.seq.isin(train.seq)]
test_2019.reset_index(drop=True, inplace=True)

test_2020 =test_2020[~test_2020.seq.isin(train.seq)]
test_2020.reset_index(drop=True, inplace=True)

test_2021 =test_2021[~test_2021.seq.isin(train.seq)]
test_2021.reset_index(drop=True, inplace=True)

test_2022 =test_2022[~test_2022.seq.isin(train.seq)]
test_2022.reset_index(drop=True, inplace=True)

In [45]:
test_2022

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
0,P02812,False,False,0,-,0,1986-07-21,MLLILLSVALLALSSAQNLNEDVSQEESPSLIAGNPQGAPPQGGNK...,416
1,P02883,False,False,0,-,0,1986-07-21,MAATTCFFFLFPFLLLLTLSRAATFEIVNRCSYTVWAAASKGDAAL...,235
2,P01160,False,False,0,-,0,1986-07-21,MSSFSTTTVSFLLLLAFQLLGQTRANPMYNAVSNADLMDFKNLLDH...,151
3,P00780,True,False,1,3.4.21.62,4,1986-07-21,MMRKKSFWLGMLTAFMLVFTMAFSDSASAAQPAKNVEKDYIVGFKS...,379
4,P01523,False,False,0,-,0,1986-07-21,MMSKLGVLLTICLLLFPLTALPMDGDEPANRPVERMQDNISSEQYP...,75
...,...,...,...,...,...,...,...,...,...
12053,A0A2K5TU92,True,True,1,2.4.2.-; 2.3.1.286,4,2022-05-25,MSVNYAAGLSPYADKGKCGLPEIFDPPEELERKVWELARLVWQSSH...,355
12054,A0A3R0A696,True,False,1,3.2.1.55,4,2022-05-25,MKHWKKMAASLIAISTMVAVVPTTYAMESEDSQPQTTDTATVQTTK...,1065
12055,Q5ZV91,False,False,0,-,0,2022-05-25,MDEIKKDDELSQWLSTYGTITAERILGRYNISLPQDEILEAINIPS...,294
12056,M1H607,False,False,0,-,0,2022-05-25,MDAIKKKMQAMKLEKDDAMDRADTLEQQNKEANIRAEKAEEEVHNL...,284


### 5.3 Remove changed seqence in test set

In [46]:
test_2019 = test_2019[~test_2019.id.isin(test_2019.merge(train, on='id', how='inner').id.values)]
test_2019.reset_index(drop=True, inplace=True)

test_2020 = test_2020[~test_2020.id.isin(test_2020.merge(train, on='id', how='inner').id.values)]
test_2020.reset_index(drop=True, inplace=True)

test_2021 = test_2021[~test_2021.id.isin(test_2021.merge(train, on='id', how='inner').id.values)]
test_2021.reset_index(drop=True, inplace=True)

test_2022 = test_2022[~test_2022.id.isin(test_2022.merge(train, on='id', how='inner').id.values)]
test_2022.reset_index(drop=True, inplace=True)


### 5.4 Trim string

In [47]:
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2019.ec_number = test_2019.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2019.seq = test_2019.seq.parallel_apply(lambda x : str(x).strip()) #seq trim
    
    test_2020.ec_number = test_2020.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2020.seq = test_2020.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2021.ec_number = test_2021.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2021.seq = test_2021.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2022.ec_number = test_2022.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2022.seq = test_2022.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

### 5.5 Save train test

In [49]:
train.to_feather(cfg.DATADIR + 'datasets/train.feather')
test_2019.to_feather(cfg.DATADIR + 'datasets/test_2019.feather')
test_2020.to_feather(cfg.DATADIR + 'datasets/test_2020.feather')
test_2021.to_feather(cfg.DATADIR + 'datasets/test_2021.feather')
test_2022.to_feather(cfg.DATADIR + 'datasets/test_2022.feather')

## 6. Build benchmarking datasets
### 6.1 Task 1 isEnzyme

In [52]:
train = pd.read_feather(cfg.DIR_DATASETS + 'train.feather')
test_2019 = pd.read_feather(cfg.DIR_DATASETS + 'test_2019.feather')
test_2020 = pd.read_feather(cfg.DIR_DATASETS + 'test_2020.feather')
test_2021 = pd.read_feather(cfg.DIR_DATASETS + 'test_2021.feather')
test_2022 = pd.read_feather(cfg.DIR_DATASETS + 'test_2022.feather')

task1_train = train.iloc[:,np.r_[0,7,1]]

task1_test_2019 = test_2019.iloc[:,np.r_[0,7,1]]
task1_test_2020 = test_2020.iloc[:,np.r_[0,7,1]]
task1_test_2021 = test_2021.iloc[:,np.r_[0,7,1]]
task1_test_2022 = test_2022.iloc[:,np.r_[0,7,1]]


task1_train.to_feather(cfg.FILE_TASK1_TRAIN)
task1_test_2019.to_feather(cfg.FILE_TASK1_TEST_2019)
task1_test_2020.to_feather(cfg.FILE_TASK1_TEST_2020)
task1_test_2021.to_feather(cfg.FILE_TASK1_TEST_2021)
task1_test_2022.to_feather(cfg.FILE_TASK1_TEST_2022)

funclib.table2fasta(table=task1_train[['id', 'seq']], file_out=cfg.FILE_TASK1_TRAIN_FASTA)
funclib.table2fasta(table=task1_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2019_FASTA)
funclib.table2fasta(table=task1_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2020_FASTA)
funclib.table2fasta(table=task1_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2021_FASTA)
funclib.table2fasta(table=task1_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


### 6.2 Task2 Function Counts

In [53]:
task2_train = train[train.functionCounts >0]
task2_train.reset_index(drop=True, inplace=True)
task2_train = task2_train.iloc[:,np.r_[0,7,3]]

task2_test_2019 = test_2019[test_2019.functionCounts >0]
task2_test_2019.reset_index(drop=True, inplace=True)
task2_test_2019 = task2_test_2019.iloc[:,np.r_[0,7,3]]

task2_test_2020 = test_2020[test_2020.functionCounts >0]
task2_test_2020.reset_index(drop=True, inplace=True)
task2_test_2020 = task2_test_2020.iloc[:,np.r_[0,7,3]]

task2_test_2021 = test_2021[test_2021.functionCounts >0]
task2_test_2021.reset_index(drop=True, inplace=True)
task2_test_2021 = task2_test_2021.iloc[:,np.r_[0,7,3]]

task2_test_2022 = test_2022[test_2022.functionCounts >0]
task2_test_2022.reset_index(drop=True, inplace=True)
task2_test_2022 = task2_test_2022.iloc[:,np.r_[0,7,3]]

task2_train.to_feather(cfg.FILE_TASK2_TRAIN)
task2_test_2019.to_feather(cfg.FILE_TASK2_TEST_2019)
task2_test_2020.to_feather(cfg.FILE_TASK2_TEST_2020)
task2_test_2021.to_feather(cfg.FILE_TASK2_TEST_2021)
task2_test_2022.to_feather(cfg.FILE_TASK2_TEST_2022)

funclib.table2fasta(table=task2_train[['id', 'seq']], file_out=cfg.FILE_TASK2_TRAIN_FASTA)
funclib.table2fasta(table=task2_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2019_FASTA)
funclib.table2fasta(table=task2_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2020_FASTA)
funclib.table2fasta(table=task2_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2021_FASTA)
funclib.table2fasta(table=task2_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


### 6.3 Task3 EC Number

In [54]:
task3_train = train[train.functionCounts >0]
task3_train.reset_index(drop=True, inplace=True)
task3_train = task3_train.iloc[:,np.r_[0,7,4]]

task3_test_2019 = test_2019[test_2019.functionCounts >0]
task3_test_2019.reset_index(drop=True, inplace=True)
task3_test_2019 = task3_test_2019.iloc[:,np.r_[0,7,4]]

task3_test_2020 = test_2020[test_2020.functionCounts >0]
task3_test_2020.reset_index(drop=True, inplace=True)
task3_test_2020 = task3_test_2020.iloc[:,np.r_[0,7,4]]

task3_test_2021 = test_2021[test_2021.functionCounts >0]
task3_test_2021.reset_index(drop=True, inplace=True)
task3_test_2021 = task3_test_2021.iloc[:,np.r_[0,7,4]]

task3_test_2022 = test_2022[test_2022.functionCounts >0]
task3_test_2022.reset_index(drop=True, inplace=True)
task3_test_2022 = task3_test_2022.iloc[:,np.r_[0,7,4]]

task3_train.to_feather(cfg.FILE_TASK3_TRAIN)
task3_test_2019.to_feather(cfg.FILE_TASK3_TEST_2019)
task3_test_2020.to_feather(cfg.FILE_TASK3_TEST_2020)
task3_test_2021.to_feather(cfg.FILE_TASK3_TEST_2021)
task3_test_2022.to_feather(cfg.FILE_TASK3_TEST_2022)

funclib.table2fasta(table=task3_train[['id', 'seq']], file_out=cfg.FILE_TASK3_TRAIN_FASTA)
funclib.table2fasta(table=task3_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2019_FASTA)
funclib.table2fasta(table=task3_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2020_FASTA)
funclib.table2fasta(table=task3_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2021_FASTA)
funclib.table2fasta(table=task3_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


## 7 Make Feature Bank

### 7.1 ESM embedding 

In [9]:
# loading sprot data
snap18 = pd.read_feather(cfg.DIR_UNIPROT + '\\snap201802.feather')
snap19 = pd.read_feather(cfg.DIR_UNIPROT + '\\snap201902.feather')
snap20 = pd.read_feather(cfg.DIR_UNIPROT + '\\snap202006.feather')
snap21 = pd.read_feather(cfg.DIR_UNIPROT + '\\snap202102.feather')
snap22 = pd.read_feather(cfg.DIR_UNIPROT + '\\snap202202.feather')
# merge
full_snap_data = pd.concat([snap18, snap19,snap20,snap21,snap22], axis=0)
full_snap_data = full_snap_data.sort_values(by=['id', 'date_annotation_update'], ascending=False)
full_snap_data = full_snap_data[['id', 'seq']].drop_duplicates(subset='id', keep='first')
full_snap_data.reset_index(drop=True, inplace=True)


# loading exsisting features
if ftool.isfileExists(cfg.FILE_FEATURE_ESM0):
    feature_esm0 = pd.read_feather(cfg.FILE_FEATURE_ESM0)
    feature_esm32 = pd.read_feather(cfg.FILE_FEATURE_ESM32)
    feature_esm33 = pd.read_feather(cfg.FILE_FEATURE_ESM33)
    feature_unirep = pd.read_feather(cfg.FILE_FEATURE_UNIREP)
    feature_onehot = pd.read_feather(cfg.FILE_FEATURE_ONEHOT)
    #caculate embedding list
    needesm = full_snap_data[~full_snap_data.id.isin(list(set(feature_esm33.id)))]
    needunirep = full_snap_data[~full_snap_data.id.isin(list(set(feature_unirep.id)))]
    needonehot = full_snap_data[~full_snap_data.id.isin(list(set(feature_onehot.id)))]
else:
    needesm = full_snap_data
    needunirep = full_snap_data
    needonehot = full_snap_data



In [10]:
#!pip install fair-esm
from tools import embedding_esm as esmebd
if len(needesm)>0:
    tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=needesm, model='esm1b_t33_650M_UR50S',seqthres=1022)

    #merge existing
    print("1")
    feature_esm0 = pd.concat([feature_esm0, tr_rep0], axis=0).reset_index(drop=True)
    print("2")    
    feature_esm32 = pd.concat([feature_esm32, tr_rep32], axis=0).reset_index(drop=True)
    print("3")
    feature_esm33 = pd.concat([feature_esm33, tr_rep33], axis=0).reset_index(drop=True)


    #save
    print("4")
    feature_esm0.to_feather(cfg.FILE_FEATURE_ESM0)
    print("5")
    feature_esm32.to_feather(cfg.FILE_FEATURE_ESM32)
    print("6")
    feature_esm33.to_feather(cfg.FILE_FEATURE_ESM33)

Transferred model to GPU


  0%|          | 123/500707 [00:57<65:07:51,  2.13it/s] 


OutOfMemoryError: CUDA out of memory. Tried to allocate 720.00 MiB (GPU 0; 5.00 GiB total capacity; 3.54 GiB already allocated; 0 bytes free; 4.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
import torch
print(torch. __version__)

1.13.0+cu117


### 7.2 Unirep

In [4]:
if len(needunirep) > 0:
    from tools import embedding_unirep as unirep
    tr_unirep = unirep.getunirep(needunirep, 40)

    feature_unirep = pd.concat([feature_unirep, tr_unirep],axis=0).reset_index(drop=True)
    feature_unirep.to_feather(cfg.FILE_FEATURE_UNIREP)


100%|██████████| 49/49 [14:24<00:00, 17.65s/it]

length not match


### 7.3 one-hot

In [10]:
feature_onehot = pd.DataFrame()
needonehot = full_snap_data
if len(needonehot) > 0:
    tr_unirep = onehotebd.get_onehot(sequences=needonehot, padding=True, padding_window=1500)
    feature_onehot = pd.concat([feature_onehot, tr_unirep],axis=0).reset_index(drop=True)
    feature_onehot.to_feather(cfg.FILE_FEATURE_ONEHOT)

In [ ]:
import xml.etree.ElementTree as et 
pd.read_xml(f'{cfg.DATADIR}explorenz/enzyme-data.xml')
xtree = et.parse(f'{cfg.DATADIR}explorenz/enzyme-data.xml')
xroot = xtree.getroot()

counter = 1
for node in xroot:
    print(dir(node))
    print(node.attrib.get('row'))

    counter = counter +1
    if counter ==15:
        break